In [ ]:
import openai
import json
import time
import random
import threading
from google.colab import userdata

# Set your OpenAI API key here
openai.api_key = userdata.get('OPENAI_API_KEY')

# client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
# Set up your OpenAI API key (stored securely in Colab secrets)
# openai.api_key = userdata.get('OPENAI_API_KEY')

In [ ]:
class Directory:
    restaurants = []

    @classmethod
    def register(cls, restaurant):
        cls.restaurants.append(restaurant)

    @classmethod
    def get_restaurants(cls):
        return cls.restaurants

class Restaurant:
    def __init__(self, name, container):
        self.name = name
        self.container = container
        self.is_free = True
        self.memory = []
        self.update_system_prompt()

    def update_system_prompt(self):
        state = "free" if self.is_free else "occupied"
        instructions = f"You are a restaurant agent named {self.name}. Your container is {self.container}. Current state: {state}.\n"
        instructions += "Respond only with JSON like {'performative': 'ACCEPT_PROPOSAL', 'content': '...'}\n"
        instructions += "For a PROPOSE message with content 'I'd like to book', if free, accept with content being your container name, and you will become occupied after.\n"
        instructions += "If occupied, reject with {'performative': 'REJECT_PROPOSAL', 'content': 'Sorry :/'}\n"
        instructions += "For a CONFIRM message, respond with {'performative': 'INFORM', 'content': 'Welcome'} and note the arrival.\n"
        self.system_prompt = {"role": "system", "content": instructions}

    def handle_message(self, sender, performative, content):
        message = f"From {sender}: performative {performative}, content '{content}'"
        self.memory.append({"role": "user", "content": message})
        messages = [self.system_prompt] + self.memory
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            response_format={"type": "json_object"}
        )
        resp_str = response.choices[0].message.content
        self.memory.append({"role": "assistant", "content": resp_str})
        resp = json.loads(resp_str)
        if resp['performative'] == 'ACCEPT_PROPOSAL':
            self.is_free = False
            self.update_system_prompt()
        if resp['performative'] == 'INFORM' and performative == 'CONFIRM':
            print(f"({self.name}) confirms arrival of {sender}")
        return resp

class Customer:
    def __init__(self, name):
        self.name = name
        self.delay = random.randint(15, 50)
        self.memory = []
        self.system_prompt = {
            "role": "system",
            "content": "You are a customer agent looking to book a restaurant.\n"
                       "Actions are JSON objects.\n"
                       "To propose: {'action': 'send', 'receiver': 'restaurant_name', 'performative': 'PROPOSE', 'content': 'I'd like to book'}\n"
                       "On ACCEPT_PROPOSAL reply (content is container): {'action': 'move', 'restaurant': 'the_restaurant', 'container': 'the_container'}\n"
                       "After moving: {'action': 'send', 'receiver': 'the_restaurant', 'performative': 'CONFIRM', 'content': 'Arrived'}\n"
                       "After confirming: {'action': 'done'}\n"
                       "On REJECT_PROPOSAL: propose to the next restaurant not yet tried.\n"
                       "If no more restaurants: {'action': 'no_restaurant'}\n"
                       "Keep track of tried restaurants and the list in history. Respond only with the JSON action."
        }
        self.found = False
        self.restaurants = []

    def get_next_action(self, additional_info):
        self.memory.append({"role": "user", "content": additional_info})
        messages = [self.system_prompt] + self.memory
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            response_format={"type": "json_object"}
        )
        resp_str = response.choices[0].message.content
        self.memory.append({"role": "assistant", "content": resp_str})
        return json.loads(resp_str)

    def start(self):
        time.sleep(self.delay)
        print(f"Customer {self.name} appears and is searching for restaurants")
        self.restaurants = Directory.get_restaurants()
        if not self.restaurants:
            print(f"[{self.name}] No restaurants found")
            return
        info = f"List of restaurants: {', '.join(r.name for r in self.restaurants)}. Start by proposing to the first one."
        action = self.get_next_action(info)
        current_restaurant = None
        while True:
            if action['action'] == 'send':
                restaurant = next((r for r in self.restaurants if r.name == action['receiver']), None)
                if restaurant is None:
                    print(f"[{self.name}] Invalid restaurant {action['receiver']}")
                    break
                print(f"[{self.name}] proposing to {restaurant.name}")
                reply = restaurant.handle_message(self.name, action['performative'], action['content'])
                info = f"Received from {action['receiver']}: performative {reply['performative']}, content '{reply['content']}'"
                action = self.get_next_action(info)
            elif action['action'] == 'move':
                current_restaurant = action['restaurant']
                print(f"[{self.name}] found restaurant {current_restaurant}")
                print(f"[{self.name}] moved to container {action['container']}")
                info = "You have moved to the container. Now send the confirmation to the restaurant."
                action = self.get_next_action(info)
            elif action['action'] == 'no_restaurant':
                print(f"[{self.name}] checked all restaurants but all declined")
                break
            elif action['action'] == 'done':
                print(f"[{self.name}] eating at restaurant")
                self.found = True
                break
            else:
                print(f"[{self.name}] unknown action {action['action']}")
                break

In [ ]:
if __name__ == "__main__":
    # Create some restaurants
    restaurant1 = Restaurant("Restaurant1", "Container1")
    Directory.register(restaurant1)
    restaurant2 = Restaurant("Restaurant2", "Container2")
    Directory.register(restaurant2)

    # Create customers in threads to simulate concurrency
    customer1 = Customer("Customer1")
    thread1 = threading.Thread(target=customer1.start)
    thread1.start()

    customer2 = Customer("Customer2")
    thread2 = threading.Thread(target=customer2.start)
    thread2.start()

    thread1.join()
    thread2.join()

Customer Customer2 appears and is searching for restaurants
[Customer2] proposing to Restaurant1
[Customer2] found restaurant Restaurant1
[Customer2] moved to container Container1
Customer Customer1 appears and is searching for restaurants
[Customer2] proposing to Restaurant1
[Customer1] proposing to Restaurant1
(Restaurant1) confirms arrival of Customer2
[Customer2] eating at restaurant
[Customer1] proposing to Restaurant2
[Customer1] found restaurant Restaurant2
[Customer1] moved to container Container2
[Customer1] proposing to Restaurant2
(Restaurant2) confirms arrival of Customer1
[Customer1] eating at restaurant
